<a href="https://colab.research.google.com/github/nocturnalrrx/portfolio/blob/main/task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Названия столбцов: <br>
*us_id* – user id <br>
*event_name* – название ивента <br>
*created_at* – время ивента <br>
*installed_at* - время установки приложения <br>

Содержит ивенты: <br>
*ad_start_session* - Старт сессии (зашел в приложение) <br>
*ad_banner_impression* - Показ баннерной рекламы <br>
*ad_interstitial_impression* - Показ inter рекламы <br>

1. Рассчитать DAU для каждого дня
2. Рассчитать накопительные показы на юзера по типам рекламы. Данные должны быть в виде таблицы, где названия строк — это дата установки, а названия столбцов — день жизни когорты. 2й столбец таблицы - Кол-во юзеров в когорте  
3. Рассчитать Retention rate до максимального дня жизни когорты. Данные должны быть в виде таблицы, где названия строк — это дата установки, а названия столбцов — день жизни когорты. 2й столбец таблицы - Кол-во юзеров в когорте


In [86]:
import pandas as pd
import plotly.express as px
import gdown

pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.max_columns', None)

In [87]:
file_id = '1Fmq-MIxnzPZqftZH1PpDY0732tcGN2Cm'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'dataset.csv'  # Name to save the file as

gdown.download(url, output, quiet=False)

# Read the downloaded file into a Pandas DataFrame
df = pd.read_csv(output)

Downloading...
From (original): https://drive.google.com/uc?id=1Fmq-MIxnzPZqftZH1PpDY0732tcGN2Cm
From (redirected): https://drive.google.com/uc?id=1Fmq-MIxnzPZqftZH1PpDY0732tcGN2Cm&confirm=t&uuid=847efde9-497d-436b-bf0c-96ae9f441ecc
To: /content/dataset.csv
100%|██████████| 331M/331M [00:02<00:00, 125MB/s]


In [88]:
df.head()

,Unnamed: 0,us_id,event_name,created_at,installed_at
0,0,4C7FF05C7B7386918022,ad_banner_impression,1527035110,1526425759
1,1,4C7FF05C7B7386918022,ad_banner_impression,1527035125,1526425759
2,2,4C7FF05C7B7386918022,ad_banner_impression,1527035140,1526425759
3,3,4C7FF05C7B7386918022,ad_banner_impression,1527035154,1526425759
4,4,4C7FF05C7B7386918022,ad_banner_impression,1527035171,1526425759


In [89]:
df.describe()

,Unnamed: 0,created_at,installed_at
count,4.613513e+06,4.613513e+06,4.613513e+06
mean,2.306756e+06,1.526591e+09,1.525789e+09
std,1.331807e+06,7.780376e+05,4.321364e+05
min,0.000000e+00,1.525133e+09,1.525133e+09
25%,1.153378e+06,1.525963e+09,1.525396e+09
50%,2.306756e+06,1.526529e+09,1.525737e+09
75%,3.460134e+06,1.527283e+09,1.526166e+09
max,4.613512e+06,1.527984e+09,1.526602e+09


In [90]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4613513 entries, 0 to 4613512
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   Unnamed: 0    4613513 non-null  int64 
 1   us_id         4613513 non-null  object
 2   event_name    4613513 non-null  object
 3   created_at    4613513 non-null  int64 
 4   installed_at  4613513 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 176.0+ MB


In [91]:
df = df.drop('Unnamed: 0', axis=1)
df.created_at = pd.to_datetime(df.created_at, unit='s')
df.installed_at = pd.to_datetime(df.installed_at, unit='s')

In [92]:
df.head()

,us_id,event_name,created_at,installed_at
0,4C7FF05C7B7386918022,ad_banner_impression,2018-05-23 00:25:10,2018-05-15 23:09:19
1,4C7FF05C7B7386918022,ad_banner_impression,2018-05-23 00:25:25,2018-05-15 23:09:19
2,4C7FF05C7B7386918022,ad_banner_impression,2018-05-23 00:25:40,2018-05-15 23:09:19
3,4C7FF05C7B7386918022,ad_banner_impression,2018-05-23 00:25:54,2018-05-15 23:09:19
4,4C7FF05C7B7386918022,ad_banner_impression,2018-05-23 00:26:11,2018-05-15 23:09:19


In [93]:
#Посмотрим min и max даты
df.describe()

,created_at,installed_at
count,4613513,4613513
mean,2018-05-17 20:57:43.081429760,2018-05-08 14:19:57.193039104
min,2018-05-01 00:00:23,2018-05-01 00:00:24
25%,2018-05-10 14:29:42,2018-05-04 01:00:52
50%,2018-05-17 03:45:37,2018-05-07 23:48:03
75%,2018-05-25 21:12:41,2018-05-12 23:04:39
max,2018-06-02 23:59:59,2018-05-17 23:59:34


In [94]:
df.sort_values(by='created_at', ascending=True, inplace=True)

In [95]:
temp = df.loc[(df['created_at']>='2018-05-18') & (df['created_at']<'2018-06-02')]

In [96]:
DAU = temp.groupby(temp['created_at'].dt.date)['us_id'].nunique().reset_index()
DAU.columns = ['date', 'users']

Получаем DAU

In [97]:
DAU

,date,users
0,2018-05-18,1815
1,2018-05-19,1540
2,2018-05-20,1475
3,2018-05-21,1411
4,2018-05-22,1408
5,2018-05-23,1350
6,2018-05-24,1248
7,2018-05-25,1230
8,2018-05-26,1130
9,2018-05-27,1121


In [98]:
fig = px.line(DAU, x='date', y='users')
fig.update_layout(title='Daily Active Users', xaxis_title='Date', yaxis_title='DAU')
fig.show()

**Теперь посчитаем показы рекламы**

Сначала создадим датафрейм, отфильтрованный по нужным датам и событиям. <br>
И посчитаем через сколько дней после установки юзер посмотрел рекламу.

In [99]:
impressions = df.copy()
impressions = impressions.loc[(impressions['created_at']>='2018-05-18') & (impressions['created_at']<'2018-06-02') &
                              impressions['event_name'].isin(['ad_banner_impression', 'ad_interstitial_impression'])]
impressions['installed_at_date'] = impressions['installed_at'].dt.date
impressions.set_index('installed_at_date', inplace=True)
impressions.sort_values(by='installed_at', ascending=True, inplace=True)
impressions['days_since_install'] = (impressions['created_at'] - impressions['installed_at']).dt.days

In [100]:
impressions.head()

,us_id,event_name,created_at,installed_at,days_since_install
installed_at_date,,,,,
2018-05-01,D40182B4E81B0BF36854,ad_banner_impression,2018-05-28 14:11:11,2018-05-01 00:00:24,27
2018-05-01,D40182B4E81B0BF36854,ad_banner_impression,2018-05-31 18:56:20,2018-05-01 00:00:24,30
2018-05-01,D40182B4E81B0BF36854,ad_banner_impression,2018-05-31 18:55:47,2018-05-01 00:00:24,30
2018-05-01,D40182B4E81B0BF36854,ad_banner_impression,2018-05-31 18:55:32,2018-05-01 00:00:24,30
2018-05-01,D40182B4E81B0BF36854,ad_banner_impression,2018-05-31 18:55:17,2018-05-01 00:00:24,30


**Напишем функцию для преобразования датафрейма** <br>
- Сначала фильтруем датафрейм с показами рекламы *ad_type_df* по нужному типу рекламы.<br>
- Считаем размеры когорт и кладем в отдельный датафрейм *ad_type_cohort_size*.<br>
- Пивотим *ad_type_df*.<br>
- К *ad_type_cohort_size* джойним *ad_type_df*, убираем NaN, переименовываем столбцы после apply, приводим значения к int.<br>
- Применим функцию, указав датафрейм с показами рекламы и нужный тип рекламы, и получим нужные по заданию таблицы.<br>

In [101]:
def pivot_by_ad_type(df:pd.DataFrame, ad_type:str) -> pd.DataFrame:
    ad_type_df = df.loc[df['event_name']==ad_type]
    ad_type_cohort_size = ad_type_df.groupby('installed_at_date')['us_id'].nunique().reset_index()
    ad_type_pivot = ad_type_df.pivot_table(index='installed_at_date', columns='days_since_install', values='event_name', aggfunc='count')
    ad_type_pivot_join = pd.merge(ad_type_cohort_size, ad_type_pivot, how='inner', on='installed_at_date')
    ad_type_pivot_join.rename(columns={'us_id': 'cohort_size'}, inplace=True)
    ad_type_pivot_join.set_index('installed_at_date', inplace=True)
    ad_type_pivot_join.index.name = 'installed_at'
    ad_type_pivot_join = ad_type_pivot_join.apply(lambda x: pd.Series(x.dropna().values), axis=1)
    ad_type_pivot_join.columns = ['cohort_size'] + [f'day {_}' for _ in range(1, len(ad_type_pivot_join.columns))]
    ad_type_pivot_join = ad_type_pivot_join.astype('Int64')
    return ad_type_pivot_join

banner = pivot_by_ad_type(impressions, 'ad_banner_impression')
inter = pivot_by_ad_type(impressions, 'ad_interstitial_impression')

In [102]:
banner

,cohort_size,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,day 10,day 11,day 12,day 13,day 14,day 15,day 16
installed_at,,,,,,,,,,,,,,,,,
2018-05-01,162,4168,8060,6367,5909,4861,4352,4507,6461,7591,8160,8341,5864,8361,7842,9521,3333
2018-05-02,133,2341,3866,4499,4411,3928,3605,3588,2919,5326,6292,6795,6294,5944,5202,5314,1963
2018-05-03,166,2955,7205,8507,9203,7973,10237,9483,7576,8690,9374,10162,11558,7305,10267,8876,5828
2018-05-04,156,2413,4216,6126,5550,6230,5851,8197,7300,7376,6014,9671,11916,8843,9108,7692,4569
2018-05-05,158,2241,6206,6394,5767,3787,5759,4890,6854,5483,8500,7283,4905,5598,6263,6882,3799
2018-05-06,173,2970,8449,7115,7168,7349,6965,6264,6514,7262,9157,9743,7239,8589,9338,9806,4461
2018-05-07,151,4491,4769,5959,6991,5648,5287,5705,4839,6144,6792,6817,6561,5982,7024,5913,2921
2018-05-08,149,4469,4106,4576,4749,6482,7370,5002,4546,4469,4266,5939,6777,7859,5348,5758,2368
2018-05-09,171,3292,6509,5756,5557,8052,5338,4825,7122,7622,4581,5321,5101,4630,5552,6241,3929


In [103]:
inter

,cohort_size,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,day 10,day 11,day 12,day 13,day 14,day 15,day 16
installed_at,,,,,,,,,,,,,,,,,
2018-05-01,135,162,249,240,255,142,178,171,182,213,226,214,179,279,211,271,91
2018-05-02,109,95,171,173,162,175,178,130,117,183,206,212,167,175,175,158,74
2018-05-03,137,109,251,221,273,228,314,272,193,178,208,236,228,173,213,199,117
2018-05-04,125,96,186,192,202,251,244,336,221,235,148,219,316,247,248,216,132
2018-05-05,132,99,217,233,219,143,185,168,218,168,221,177,133,150,148,206,127
2018-05-06,152,121,250,224,207,231,250,222,211,203,220,238,173,215,232,233,112
2018-05-07,129,114,166,202,226,201,162,149,150,126,135,124,155,157,127,127,77
2018-05-08,124,213,168,162,146,173,206,210,144,112,123,161,168,196,124,155,57
2018-05-09,140,111,250,211,217,220,200,196,216,197,152,192,148,130,169,165,134


**Теперь посчитаем retention**

In [104]:
ret_df = df.copy()
ret_df['installed_at_date'] = ret_df['installed_at'].dt.date
ret_df.set_index('installed_at_date', inplace=True)
ret_df.sort_values(by='installed_at', ascending=True, inplace=True)
ret_df['days_since_install'] = (ret_df['created_at'] - ret_df['installed_at']).dt.days

In [105]:
def pivot_by_unique_users(df:pd.DataFrame) -> pd.DataFrame:
    cohort_size = df.groupby('installed_at_date')['us_id'].nunique().reset_index()
    pivot = df.pivot_table(index='installed_at_date', columns='days_since_install', values='us_id', aggfunc='nunique')
    pivot_join = pd.merge(cohort_size, pivot, how='left', on='installed_at_date')
    pivot_join.rename(columns={'us_id': 'cohort_size'}, inplace=True)
    pivot_join.set_index('installed_at_date', inplace=True)
    pivot_join.index.name = 'installed_at'
    #pivot_join = pivot_join.fillna('0')
    #pivot_join = pivot_join.apply(lambda x: x.astype(int) if x.notnull().all() else x)
    #pivot_join.columns = ['cohort_size'] + [f'day {_}' for _ in range(1, len(pivot_join.columns))]
    #pivot_join = pivot_join.astype(int)
    return pivot_join

In [106]:
pivot_by_unique_users(ret_df)

,cohort_size,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
installed_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-05-01,599,204.0,571.0,221.0,197.0,174.0,141.0,145.0,126.0,109.0,119.0,111.0,114.0,110.0,100.0,99.0,99.0,90.0,91.0,86.0,85.0,75.0,75.0,76.0,70.0,75.0,70.0,78.0,73.0,73.0,65.0,68.0,74.0,59.0,45.0
2018-05-02,572,209.0,547.0,198.0,159.0,124.0,124.0,101.0,110.0,92.0,96.0,91.0,65.0,71.0,67.0,72.0,66.0,67.0,64.0,58.0,51.0,60.0,52.0,51.0,55.0,55.0,49.0,59.0,42.0,51.0,47.0,46.0,51.0,20.0,NaN
2018-05-03,504,182.0,482.0,205.0,151.0,152.0,139.0,143.0,131.0,111.0,108.0,100.0,95.0,95.0,97.0,99.0,104.0,81.0,70.0,97.0,91.0,87.0,80.0,82.0,75.0,71.0,73.0,76.0,61.0,64.0,58.0,66.0,50.0,NaN,NaN
2018-05-04,455,166.0,424.0,155.0,135.0,130.0,117.0,109.0,98.0,82.0,81.0,76.0,83.0,86.0,78.0,79.0,79.0,77.0,64.0,62.0,60.0,60.0,68.0,62.0,62.0,63.0,68.0,61.0,62.0,61.0,60.0,38.0,NaN,NaN,NaN
2018-05-05,533,191.0,512.0,198.0,167.0,145.0,121.0,118.0,106.0,99.0,101.0,88.0,90.0,75.0,76.0,86.0,76.0,72.0,65.0,73.0,63.0,63.0,58.0,68.0,59.0,54.0,58.0,61.0,57.0,52.0,41.0,NaN,NaN,NaN,NaN
2018-05-06,583,206.0,553.0,198.0,160.0,144.0,124.0,123.0,111.0,106.0,98.0,92.0,92.0,85.0,82.0,79.0,75.0,68.0,75.0,71.0,67.0,65.0,58.0,72.0,64.0,66.0,59.0,62.0,71.0,37.0,NaN,NaN,NaN,NaN,NaN
2018-05-07,513,201.0,482.0,178.0,161.0,148.0,125.0,105.0,110.0,116.0,109.0,100.0,81.0,77.0,65.0,79.0,73.0,66.0,78.0,59.0,58.0,53.0,62.0,64.0,62.0,63.0,60.0,61.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-08,502,176.0,468.0,186.0,142.0,128.0,104.0,102.0,100.0,91.0,84.0,78.0,65.0,73.0,57.0,72.0,71.0,64.0,63.0,45.0,47.0,46.0,54.0,56.0,44.0,44.0,35.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-09,541,194.0,504.0,178.0,134.0,118.0,106.0,115.0,94.0,100.0,93.0,77.0,72.0,72.0,81.0,64.0,68.0,64.0,54.0,50.0,49.0,50.0,52.0,50.0,48.0,45.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Видим столбец '-1'. Похоже, что-то не так со временем.
Изучим строки, где days_since_install == -1

In [107]:
ret_df.loc[ret_df['days_since_install']==-1]

,us_id,event_name,created_at,installed_at,days_since_install
installed_at_date,,,,,
2018-05-01,D40182B4E81B0BF36854,ad_start_session,2018-05-01 00:00:23,2018-05-01 00:00:24,-1
2018-05-01,ADC8CE6E8BC593703743,ad_start_session,2018-05-01 00:01:14,2018-05-01 00:01:15,-1
2018-05-01,A8B9F322C25421393CAC,ad_start_session,2018-05-01 00:03:31,2018-05-01 00:03:32,-1
2018-05-01,F6288420709B249EAF9E,ad_start_session,2018-05-01 00:04:58,2018-05-01 00:04:59,-1
2018-05-01,F55F6564F6AC884127EA,ad_start_session,2018-05-01 00:06:31,2018-05-01 00:06:34,-1
...,...,...,...,...,...
2018-05-17,BD481F347DA03EED5CE0,ad_start_session,2018-05-17 23:36:53,2018-05-17 23:36:55,-1
2018-05-17,182C1B599FB8F07B86EE,ad_start_session,2018-05-17 23:40:49,2018-05-17 23:40:50,-1
2018-05-17,AE78B213E051D4953D09,ad_start_session,2018-05-17 23:55:08,2018-05-17 23:55:12,-1


In [108]:
days_below_zero = ret_df.loc[ret_df['days_since_install']==-1]

In [109]:
(days_below_zero['installed_at']-days_below_zero['created_at']).describe()

,0
count,3183
mean,0 days 00:00:02.633993088
std,0 days 00:00:04.848120734
min,0 days 00:00:01
25%,0 days 00:00:01
50%,0 days 00:00:01
75%,0 days 00:00:02
max,0 days 00:00:54


Получили 3200 таких строк.<br>
Видим, что в этих случаях события происходят раньше установки. <br>
Вряд ли такое возможно :) Скорее всего по какой-то причине время установки записалось позже времени события.<br>
<br>
В основном разница между временем события и временем установки составляет около 2 секунд.<br>
В данном случае заменю нулями все days_since_install равные '-1'<br>


In [110]:
ret_df['days_since_install'] = ret_df['days_since_install'].replace(-1, 0)

In [111]:
pivot_by_unique_users(ret_df)

,cohort_size,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
installed_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-05-01,599,591.0,221.0,197.0,174.0,141.0,145.0,126.0,109.0,119.0,111.0,114.0,110.0,100.0,99.0,99.0,90.0,91.0,86.0,85.0,75.0,75.0,76.0,70.0,75.0,70.0,78.0,73.0,73.0,65.0,68.0,74.0,59.0,45.0
2018-05-02,572,567.0,198.0,159.0,124.0,124.0,101.0,110.0,92.0,96.0,91.0,65.0,71.0,67.0,72.0,66.0,67.0,64.0,58.0,51.0,60.0,52.0,51.0,55.0,55.0,49.0,59.0,42.0,51.0,47.0,46.0,51.0,20.0,NaN
2018-05-03,504,496.0,205.0,151.0,152.0,139.0,143.0,131.0,111.0,108.0,100.0,95.0,95.0,97.0,99.0,104.0,81.0,70.0,97.0,91.0,87.0,80.0,82.0,75.0,71.0,73.0,76.0,61.0,64.0,58.0,66.0,50.0,NaN,NaN
2018-05-04,455,445.0,155.0,135.0,130.0,117.0,109.0,98.0,82.0,81.0,76.0,83.0,86.0,78.0,79.0,79.0,77.0,64.0,62.0,60.0,60.0,68.0,62.0,62.0,63.0,68.0,61.0,62.0,61.0,60.0,38.0,NaN,NaN,NaN
2018-05-05,533,531.0,198.0,167.0,145.0,121.0,118.0,106.0,99.0,101.0,88.0,90.0,75.0,76.0,86.0,76.0,72.0,65.0,73.0,63.0,63.0,58.0,68.0,59.0,54.0,58.0,61.0,57.0,52.0,41.0,NaN,NaN,NaN,NaN
2018-05-06,583,579.0,198.0,160.0,144.0,124.0,123.0,111.0,106.0,98.0,92.0,92.0,85.0,82.0,79.0,75.0,68.0,75.0,71.0,67.0,65.0,58.0,72.0,64.0,66.0,59.0,62.0,71.0,37.0,NaN,NaN,NaN,NaN,NaN
2018-05-07,513,506.0,178.0,161.0,148.0,125.0,105.0,110.0,116.0,109.0,100.0,81.0,77.0,65.0,79.0,73.0,66.0,78.0,59.0,58.0,53.0,62.0,64.0,62.0,63.0,60.0,61.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-08,502,494.0,186.0,142.0,128.0,104.0,102.0,100.0,91.0,84.0,78.0,65.0,73.0,57.0,72.0,71.0,64.0,63.0,45.0,47.0,46.0,54.0,56.0,44.0,44.0,35.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-09,541,534.0,178.0,134.0,118.0,106.0,115.0,94.0,100.0,93.0,77.0,72.0,72.0,81.0,64.0,68.0,64.0,54.0,50.0,49.0,50.0,52.0,50.0,48.0,45.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
ret_df = pivot_by_unique_users(ret_df)
ret_df.columns = ['cohort_size'] + [f'day {_}' for _ in range(1, len(ret_df.columns))]

In [113]:
ret_df

,cohort_size,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,day 10,day 11,day 12,day 13,day 14,day 15,day 16,day 17,day 18,day 19,day 20,day 21,day 22,day 23,day 24,day 25,day 26,day 27,day 28,day 29,day 30,day 31,day 32,day 33
installed_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-05-01,599,591.0,221.0,197.0,174.0,141.0,145.0,126.0,109.0,119.0,111.0,114.0,110.0,100.0,99.0,99.0,90.0,91.0,86.0,85.0,75.0,75.0,76.0,70.0,75.0,70.0,78.0,73.0,73.0,65.0,68.0,74.0,59.0,45.0
2018-05-02,572,567.0,198.0,159.0,124.0,124.0,101.0,110.0,92.0,96.0,91.0,65.0,71.0,67.0,72.0,66.0,67.0,64.0,58.0,51.0,60.0,52.0,51.0,55.0,55.0,49.0,59.0,42.0,51.0,47.0,46.0,51.0,20.0,NaN
2018-05-03,504,496.0,205.0,151.0,152.0,139.0,143.0,131.0,111.0,108.0,100.0,95.0,95.0,97.0,99.0,104.0,81.0,70.0,97.0,91.0,87.0,80.0,82.0,75.0,71.0,73.0,76.0,61.0,64.0,58.0,66.0,50.0,NaN,NaN
2018-05-04,455,445.0,155.0,135.0,130.0,117.0,109.0,98.0,82.0,81.0,76.0,83.0,86.0,78.0,79.0,79.0,77.0,64.0,62.0,60.0,60.0,68.0,62.0,62.0,63.0,68.0,61.0,62.0,61.0,60.0,38.0,NaN,NaN,NaN
2018-05-05,533,531.0,198.0,167.0,145.0,121.0,118.0,106.0,99.0,101.0,88.0,90.0,75.0,76.0,86.0,76.0,72.0,65.0,73.0,63.0,63.0,58.0,68.0,59.0,54.0,58.0,61.0,57.0,52.0,41.0,NaN,NaN,NaN,NaN
2018-05-06,583,579.0,198.0,160.0,144.0,124.0,123.0,111.0,106.0,98.0,92.0,92.0,85.0,82.0,79.0,75.0,68.0,75.0,71.0,67.0,65.0,58.0,72.0,64.0,66.0,59.0,62.0,71.0,37.0,NaN,NaN,NaN,NaN,NaN
2018-05-07,513,506.0,178.0,161.0,148.0,125.0,105.0,110.0,116.0,109.0,100.0,81.0,77.0,65.0,79.0,73.0,66.0,78.0,59.0,58.0,53.0,62.0,64.0,62.0,63.0,60.0,61.0,27.0,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-08,502,494.0,186.0,142.0,128.0,104.0,102.0,100.0,91.0,84.0,78.0,65.0,73.0,57.0,72.0,71.0,64.0,63.0,45.0,47.0,46.0,54.0,56.0,44.0,44.0,35.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-09,541,534.0,178.0,134.0,118.0,106.0,115.0,94.0,100.0,93.0,77.0,72.0,72.0,81.0,64.0,68.0,64.0,54.0,50.0,49.0,50.0,52.0,50.0,48.0,45.0,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
ret_df.fillna(0, inplace=True)

In [115]:
ret_df.iloc[:,1:] = ret_df.iloc[:,1:].div(ret_df['cohort_size'], axis=0).round(3) * 100

Смотрим наш retention

In [116]:
ret_df.style.format(precision=1).background_gradient(cmap='Blues',vmin=0,vmax=100)

,cohort_size,day 1,day 2,day 3,day 4,day 5,day 6,day 7,day 8,day 9,day 10,day 11,day 12,day 13,day 14,day 15,day 16,day 17,day 18,day 19,day 20,day 21,day 22,day 23,day 24,day 25,day 26,day 27,day 28,day 29,day 30,day 31,day 32,day 33
installed_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-05-01,599,98.7,36.9,32.9,29.0,23.5,24.2,21.0,18.2,19.9,18.5,19.0,18.4,16.7,16.5,16.5,15.0,15.2,14.4,14.2,12.5,12.5,12.7,11.7,12.5,11.7,13.0,12.2,12.2,10.9,11.4,12.4,9.8,7.5
2018-05-02,572,99.1,34.6,27.8,21.7,21.7,17.7,19.2,16.1,16.8,15.9,11.4,12.4,11.7,12.6,11.5,11.7,11.2,10.1,8.9,10.5,9.1,8.9,9.6,9.6,8.6,10.3,7.3,8.9,8.2,8.0,8.9,3.5,0.0
2018-05-03,504,98.4,40.7,30.0,30.2,27.6,28.4,26.0,22.0,21.4,19.8,18.8,18.8,19.2,19.6,20.6,16.1,13.9,19.2,18.1,17.3,15.9,16.3,14.9,14.1,14.5,15.1,12.1,12.7,11.5,13.1,9.9,0.0,0.0
2018-05-04,455,97.8,34.1,29.7,28.6,25.7,24.0,21.5,18.0,17.8,16.7,18.2,18.9,17.1,17.4,17.4,16.9,14.1,13.6,13.2,13.2,14.9,13.6,13.6,13.8,14.9,13.4,13.6,13.4,13.2,8.4,0.0,0.0,0.0
2018-05-05,533,99.6,37.1,31.3,27.2,22.7,22.1,19.9,18.6,18.9,16.5,16.9,14.1,14.3,16.1,14.3,13.5,12.2,13.7,11.8,11.8,10.9,12.8,11.1,10.1,10.9,11.4,10.7,9.8,7.7,0.0,0.0,0.0,0.0
2018-05-06,583,99.3,34.0,27.4,24.7,21.3,21.1,19.0,18.2,16.8,15.8,15.8,14.6,14.1,13.6,12.9,11.7,12.9,12.2,11.5,11.1,9.9,12.3,11.0,11.3,10.1,10.6,12.2,6.3,0.0,0.0,0.0,0.0,0.0
2018-05-07,513,98.6,34.7,31.4,28.8,24.4,20.5,21.4,22.6,21.2,19.5,15.8,15.0,12.7,15.4,14.2,12.9,15.2,11.5,11.3,10.3,12.1,12.5,12.1,12.3,11.7,11.9,5.3,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-08,502,98.4,37.1,28.3,25.5,20.7,20.3,19.9,18.1,16.7,15.5,12.9,14.5,11.4,14.3,14.1,12.7,12.5,9.0,9.4,9.2,10.8,11.2,8.8,8.8,7.0,4.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-09,541,98.7,32.9,24.8,21.8,19.6,21.3,17.4,18.5,17.2,14.2,13.3,13.3,15.0,11.8,12.6,11.8,10.0,9.2,9.1,9.2,9.6,9.2,8.9,8.3,5.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
